In [72]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://onepiece.fandom.com/fr/wiki/Chapitre_{}"
chapter_number = 1

data_frames = []

while True:
    url = base_url.format(chapter_number)
    response = requests.get(url)
    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.content, "html.parser")

    try:
        aside = soup.find("aside", class_="portable-infobox")
        if aside:
            title_element = aside.find("h2", class_="pi-item-spacing", attrs={"data-source": "nom"})
            if title_element:
                title = title_element.text.strip()
            else:
                title = ""

            img_element = aside.find("img", class_="pi-image-thumbnail")
            if img_element:
                image_url = img_element["src"]
            else:
                image_url = ""

            info_dict = {"Chapter": chapter_number, "Title": title, "Image URL": image_url}

            data_sources = ["tome", "chapitre", "nomj", "nomr", "nomf", "pages", "date", "épisode"]
            for data_source in data_sources:
                data_element = aside.find("div", class_="pi-item-spacing", attrs={"data-source": data_source})
                if data_element:
                    label_element = data_element.find("h3", class_="pi-data-label")
                    value_element = data_element.find("div", class_="pi-data-value")
                    if label_element and value_element:
                        label = label_element.text.strip()
                        value = value_element.text.strip()
                        info_dict[label] = value

            table = soup.find("table", class_="toccolours")
            if table:
                rows = table.find_all("tr")
                for row in rows[1:]:
                    cells = row.find_all("td")
                    for cell in cells:
                        dl_elements = cell.find_all("dl")
                        for dl_element in dl_elements:
                            dt_element = dl_element.find("dt")
                            if dt_element:
                                category = dt_element.find("a").text.strip()
                                ul_element = dl_element.find_next_sibling("ul")
                                if ul_element:
                                    li_elements = ul_element.find_all("li")
                                    for li_element in li_elements:
                                        character = li_element.find("a").text.strip()
                                        info_dict["Category"] = category
                                        info_dict["Character"] = character
                                        data_frames.append(pd.DataFrame([info_dict]))

    except:
        pass
    chapter_number += 1

data_OnePiece_df = pd.concat(data_frames, ignore_index=True)

In [25]:
data_OnePiece_df.to_csv('data_onePiece.csv', index=False)


NameError: name 'data_OnePiece_d' is not defined

In [27]:
df_one_piece = pd.read_csv('data_onePiece_df.csv')

In [36]:
df1 =df_one_piece.copy() 

## Ajouter resume , note etc


In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://onepiece.fandom.com/fr/wiki/Chapitre_{}"
chapter_number = 1

data = []

while True:
    url = base_url.format(chapter_number)
    response = requests.get(url)
    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.content, "html.parser")

    try:
        info_dict = {"Chapter": chapter_number,}  # Renamed 'info_dicts' to 'info_dict' to match the singular form

        # Find the Résumé Rapide
        resume_rapide_element = soup.find("span", id="Résumé_Rapide")
        if resume_rapide_element:
            resume_rapide = resume_rapide_element.find_next("p").text.strip()
            info_dict["Résumé Rapide"] = resume_rapide

        # Find the Résumés
        resumes_element = soup.find("span", id="Résumés")
        if resumes_element:
            resumes = resumes_element.find_next("p").text.strip()
            info_dict["Résumés"] = resumes

        # Find the Notes
        notes_element = soup.find("span", id="Notes")
        if notes_element:
            notes = notes_element.find_next("ul").text.strip()
            info_dict["Notes"] = notes

        # Find the Informations
        informations_element = soup.find("span", id="Informations")
        if informations_element:
            informations = informations_element.find_next("table").text.strip()
            info_dict["Informations"] = informations

        data.append(pd.DataFrame([info_dict]))  # Renamed 'info_dicts' to 'info_dict' here

    except:
        pass
    chapter_number += 1

data_One = pd.concat(data, ignore_index=True)



In [33]:
data_One

,Chapter,Résumé Rapide,Résumés,Notes,Informations
0,1,Monkey D. Luffy fait la rencontre de Shanks et...,Monkey D. Luffy fait la rencontre de Shanks et...,Premier chapitre de la première moitié du mang...,Pirates\n\nCitoyens\n\nAnimaux\n\nHors-la-loi\...
1,2,À son début de voyage pour devenir le Seigneur...,À son début de voyage pour devenir le Seigneur...,"Première apparition de Alvida, Koby, Heppoko, ...",NaN
2,3,Luffy et Koby arrivent à Shells Town une île a...,NaN,"Première Apparition de Zoro Roronoa, Hermep, R...",NaN
3,4,Luffy va à nouveau voir Zoro pour l'inviter à ...,Luffy va à nouveau voir Zoro pour l'inviter à ...,"Première Apparition de Morgan, Ririka, Rokkaku...",Pirates\n\nMarines\n\nCitoyens\n\nChasseurs de...
4,5,Morgan et ses hommes s'apprêtent à tuer Zoro e...,Morgan et ses hommes s'apprêtent à tuer Zoro e...,Première Apparition de Kuina et Kôshirô.\nPrem...,Pirates\n\nMarines\n\nCitoyens\n\n\n\nL'Équipa...
...,...,...,...,...,...
1081,1082,"Dans le journal, on apprend que T. Bone, alors...","Dans le journal, on apprend que T. Bone, alors...",Tony Tony Chopper (couverture)\nZeus (cover),"Cette section est vide, insuffisamment détaill..."
1082,1083,,,Monkey D. Dragon\nSabo\nEmporio Ivankov\nMorle...,"Cette section est vide, insuffisamment détaill..."
1083,1084,Cobra adresse au conseil des cinq doyens une r...,Cobra adresse au conseil des cinq doyens une r...,Nami (couverture)\nNico Robin (couverture)\nZe...,"Cette section est vide, insuffisamment détaill..."
1084,1085,L'ancienne reine d'Alabasta s'appellait Nefert...,L'ancienne reine d'Alabasta s'appellait Nefert...,Imu,Gouvernement Mondial\n\nArmée Révolutionnaire\...


In [34]:
df2 = data_One.copy()

In [37]:
# Convertir la colonne "Chapter" en entiers dans le premier dataframe
df1['Chapter'] = df1['Chapter'].astype(int)

# Définir l'index du deuxième dataframe sur la colonne "Chapter"
df2.set_index('Chapter', inplace=True)

# Fusionner les deux dataframes en utilisant la clé de fusion
df_merged = df1.merge(df2, left_on='Chapter', right_index=True)

# Afficher le dataframe fusionné
print(df_merged)


       Chapter                                          Title  \
0            1  Romance Dawn - À l'aube d'une grande aventure   
1            1  Romance Dawn - À l'aube d'une grande aventure   
2            1  Romance Dawn - À l'aube d'une grande aventure   
3            1  Romance Dawn - À l'aube d'une grande aventure   
4            2            Luffy, l'homme au chapeau de paille   
...        ...                                            ...   
13987     1086                     Le Conseil des Cinq Doyens   
13988     1086                     Le Conseil des Cinq Doyens   
13989     1086                     Le Conseil des Cinq Doyens   
13990     1086                     Le Conseil des Cinq Doyens   
13991     1086                     Le Conseil des Cinq Doyens   

                                               Image URL    Tome:  Chapitre:  \
0                                                    NaN        1          1   
1                                                    NaN   

In [39]:
df_merged.to_csv('Final_onePiece.csv', index=False)

In [30]:
df2.set_index('Chapter', inplace=True)


KeyError: "None of ['Chapter'] are in the columns"

In [31]:
df2

,Résumé Rapide,Résumés,Notes,Informations
0,Monkey D. Luffy fait la rencontre de Shanks et...,Monkey D. Luffy fait la rencontre de Shanks et...,Premier chapitre de la première moitié du mang...,Pirates\n\nCitoyens\n\nAnimaux\n\nHors-la-loi\...
1,À son début de voyage pour devenir le Seigneur...,À son début de voyage pour devenir le Seigneur...,"Première apparition de Alvida, Koby, Heppoko, ...",NaN
2,Luffy et Koby arrivent à Shells Town une île a...,NaN,"Première Apparition de Zoro Roronoa, Hermep, R...",NaN
3,Luffy va à nouveau voir Zoro pour l'inviter à ...,Luffy va à nouveau voir Zoro pour l'inviter à ...,"Première Apparition de Morgan, Ririka, Rokkaku...",Pirates\n\nMarines\n\nCitoyens\n\nChasseurs de...
4,Morgan et ses hommes s'apprêtent à tuer Zoro e...,Morgan et ses hommes s'apprêtent à tuer Zoro e...,Première Apparition de Kuina et Kôshirô.\nPrem...,Pirates\n\nMarines\n\nCitoyens\n\n\n\nL'Équipa...
...,...,...,...,...
1081,"Dans le journal, on apprend que T. Bone, alors...","Dans le journal, on apprend que T. Bone, alors...",Tony Tony Chopper (couverture)\nZeus (cover),"Cette section est vide, insuffisamment détaill..."
1082,,,Monkey D. Dragon\nSabo\nEmporio Ivankov\nMorle...,"Cette section est vide, insuffisamment détaill..."
1083,Cobra adresse au conseil des cinq doyens une r...,Cobra adresse au conseil des cinq doyens une r...,Nami (couverture)\nNico Robin (couverture)\nZe...,"Cette section est vide, insuffisamment détaill..."
1084,L'ancienne reine d'Alabasta s'appellait Nefert...,L'ancienne reine d'Alabasta s'appellait Nefert...,Imu,Gouvernement Mondial\n\nArmée Révolutionnaire\...


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://onepiece.fandom.com/fr/wiki/Chapitre_4"

# Envoyer une requête GET à l'URL
response = requests.get(url)

# Récupérer le contenu HTML de la réponse
html_content = response.text

# Créer un objet BeautifulSoup pour analyser le contenu HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Récupérer le Résumé Rapide
resume_rapide = soup.find('span', {'id': 'Résumé_Rapide'}).parent.text
print("Résumé Rapide :", resume_rapide)

# Récupérer les Résumés
resume_approfondi = soup.find('span', {'id': 'Résumé_Approfondi'}).parent.text
print("Résumé Approfondi :", resume_approfondi)

# Récupérer les Informations
informations = soup.find('span', {'id': 'Informations'}).parent.text
print("Informations :", informations)

# Récupérer les Notes
notes = soup.find('span', {'id': 'Notes'}).parent.find('ul').text
print("Notes :", notes)



In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


base_url = "https://onepiece.fandom.com/fr/wiki/Chapitre_{}"
chapter_number = 1

data_frame = []

while True:
    url = base_url.format(chapter_number)
    response = requests.get(url)
    if response.status_code != 200:
        break
        
    soup = BeautifulSoup(response.content,'html.parser')
    
    try:
        resume_r = soup.find('span',{'id':'Résumé_Rapide'}).parent.text
        if resume_r:
            resume_rapide= resume_r.text.strip()
        else:
            resume_rapide=""    
        resume_a = soup.find('span', {'id': 'Résumé_Approfondi'}).parent.text
        if resume_a:
            resume_approfondi = resume_a.text.strip()
        else:
            resume_approfondi = ""  
        info = soup.find('span', {'id': 'Informations'}).parent.text
        if info:
            informations = info.text.strip()
        else: 
            informations = ""
        notes = soup.find('span', {'id': 'Notes'}).parent.find('ul').text
        if notes:
            note = notes.text.stip()
        else:
            note = ""
        data_informations = {"Resume_rapide":resume_rapide,"Resume_approfondi":resume_approfondi,"Informations":informations,"notes":note}
        
        for row in rows[1:]:
            data_frame.append(pd.Dataframe([data_informations]))
    except:
        pass
    chapter_number += 1  
data_infos_df = pd.concat(data_frame,ignore_index=True)

ValueError: No objects to concatenate

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://onepiece.fandom.com/fr/wiki/Chapitre_{}"
chapter_number = 1

data_frames = []

while True:
    url = base_url.format(chapter_number)
    response = requests.get(url)
    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        resume_rapide = soup.find('span', {'id': 'Résumé_Rapide'}).find_next('p').text.strip()

        resume_approfondi_elem = soup.find('span', {'id': 'Résumé_Approfondi'})
        if resume_approfondi_elem:
            resume_approfondi = resume_approfondi_elem.find_next('p').text.strip()
        else:
            resume_approfondi = ""

        informations_elem = soup.find('span', {'id': 'Informations'})
        if informations_elem:
            informations = informations_elem.find_next('ul').text.strip()
        else:
            informations = ""

        notes_elem = soup.find('span', {'id': 'Notes'})
        if notes_elem:
            notes_list = notes_elem.find_next('ul')
            if notes_list:
                notes = notes_list.text.strip()
            else:
                notes = ""
        else:
            notes = ""

        couverture_elem = soup.find('span', {'id': 'Couverture'})
        if couverture_elem:
            couverture = couverture_elem.find_next('img')['src']
        else:
            couverture = ""

        data_informations = {
            "Résumé_rapide": resume_rapide,
            "Résumé_approfondi": resume_approfondi,
            "Informations": informations,
            "Notes": notes,
            "Couverture": couverture
        }
        data_frames.append(pd.DataFrame([data_informations]))
    except:
        pass
        chapter_number += 1

if data_frames:
    data_infos_df = pd.concat(data_frames, ignore_index=True)
else:
    data_infos_df = pd.DataFrame()







KeyboardInterrupt



In [9]:
import requests
from bs4 import BeautifulSoup

base_url = "https://onepiece.fandom.com/fr/wiki/Chapitre_{}"

# Fonction pour extraire les informations d'un chapitre
def extract_chapter_info(chapter_number):
    url = base_url.format(chapter_number)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraire les résumés
    resume_elements = soup.find_all('span', {'id': 'Résumés'})
    resumes = [element.find_next('p').text for element in resume_elements]

    # Extraire le résumé rapide
    resume_rapide_element = soup.find('span', {'id': 'Résumé_Rapide'})
    resume_rapide = resume_rapide_element.find_next('p').text

    # Extraire les notes
    notes_element = soup.find('span', {'id': 'Notes'})
    notes = [li.text for li in notes_element.find_next('ul').find_all('li')]

    # Extraire les informations
    informations_element = soup.find('span', {'id': 'Informations'})
    informations = [li.text for li in informations_element.find_next('ul').find_all('li')]

    # Retourner les informations extraites
    return {
        'resumes': resumes,
        'resume_rapide': resume_rapide,
        'notes': notes,
        'informations': informations
    }

# Récupérer les informations de tous les chapitres
all_chapter_info = []
chapter_number = 4
while True:
    chapter_info = extract_chapter_info(chapter_number)
    if chapter_info is None:
        pass  # Ignorer le chapitre s'il n'existe pas
    else:
        all_chapter_info.append(chapter_info)
    chapter_number += 1

# Afficher les informations de tous les chapitres
for chapter_number, chapter_info in enumerate(all_chapter_info, start=1):
    print("Chapitre", chapter_number)
    print("Résumés:", chapter_info['resumes'])
    print("Résumé Rapide:", chapter_info['resume_rapide'])
    print("Notes:", chapter_info['notes'])
    print("Informations:", chapter_info['informations'])
    print("------------------------")

AttributeError: 'NoneType' object has no attribute 'find_next'

## Ajouter quand un nouveau chapitre sort

In [78]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://onepiece.fandom.com/fr/wiki/Chapitre_{}"

data_OnePiece_df = pd.read_csv("data_OnePiece.csv")  # Charger le DataFrame existant à partir du fichier CSV

latest_chapter = data_OnePiece_df["Chapter"].max() if not data_OnePiece_df.empty else 0  # Obtenez le dernier chapitre du DataFrame existant

chapter_number = latest_chapter + 1  # Commencer à partir du prochain chapitre

while True:
    url = base_url.format(chapter_number)
    response = requests.get(url)
    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.content, "html.parser")

    try:
        aside = soup.find("aside", class_="portable-infobox")
        if aside:
            title_element = aside.find("h2", class_="pi-item-spacing", attrs={"data-source": "nom"})
            if title_element:
                title = title_element.text.strip()
            else:
                title = ""

            img_element = aside.find("img", class_="pi-image-thumbnail")
            if img_element:
                image_url = img_element["src"]
            else:
                image_url = ""

            info_dict = {"Chapter": chapter_number, "Title": title, "Image URL": image_url}

            data_sources = ["tome", "chapitre", "nomj", "nomr", "nomf", "pages", "date", "épisode"]
            for data_source in data_sources:
                data_element = aside.find("div", class_="pi-item-spacing", attrs={"data-source": data_source})
                if data_element:
                    label_element = data_element.find("h3", class_="pi-data-label")
                    value_element = data_element.find("div", class_="pi-data-value")
                    if label_element and value_element:
                        label = label_element.text.strip()
                        value = value_element.text.strip()
                        info_dict[label] = value

            table = soup.find("table", class_="toccolours")
            if table:
                rows = table.find_all("tr")
                for row in rows[1:]:
                    cells = row.find_all("td")
                    for cell in cells:
                        dl_elements = cell.find_all("dl")
                        for dl_element in dl_elements:
                            dt_element = dl_element.find("dt")
                            if dt_element:
                                category = dt_element.find("a").text.strip()
                                ul_element = dl_element.find_next_sibling("ul")
                                if ul_element:
                                    li_elements = ul_element.find_all("li")
                                    for li_element in li_elements:
                                        character = li_element.find("a").text.strip()
                                        info_dict["Category"] = category
                                        info_dict["Character"] = character
                                        data_OnePiece_df = data_OnePiece_df.append(info_dict, ignore_index=True)
    except:
        pass
    chapter_number += 1


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"